In [1]:
import matplotlib
matplotlib.use('Agg')

In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [3]:
from fastai.imports import *
from fastai.transforms import *
from fastai.conv_learner import *
from fastai.model import *
from fastai.dataset import *
from fastai.sgdr import *
from fastai.plots import *

In [4]:
PATH = "../data/dogscats/"
sz=224

In [45]:
??torch.multinomial

In [84]:
class WeightedRandomSampler(object):
    r"""Samples elements from [0,..,len(weights)-1] with given probabilities (weights).

    Arguments:
        weights (list)   : a list of weights, not necessary summing up to one
        num_samples (int): number of samples to draw
        replacement (bool): if ``True``, samples are drawn with replacement.
            If not, they are drawn without replacement, which means that when a
            sample index is drawn for a row, it cannot be drawn again for that row.
    """

    def __init__(self, weights, num_samples, replacement=False):
        self.weights = T(weights)
        self.num_samples = num_samples
        self.replacement = replacement

    def __iter__(self):
        return iter(torch.multinomial(self.weights, self.num_samples, self.replacement))

    def __len__(self):
        return self.num_samples

In [65]:
torch.multinomial(T([0.25, 0.75]), 5, replacement=True)


 1
 0
 1
 0
 1
[torch.cuda.LongTensor of size 5 (GPU 0)]

In [23]:
wsamp = WeightedRandomSampler([0.25, 0.75], 1, replacement=False)

n0 = 0
n1 = 0
tot = 0
for t in range(1000):
    for w in wsamp:
        if w == 0:
            n0 += 1
        elif w == 1:
            n1 += 1
        tot += 1
assert tot == n0 + n1
print(n0/tot)
print(n1/tot)

0.246
0.754


In [143]:
def check_ws(ws, ds):
    n0 = 0
    n1 = 0
    tot = 0

    for w in tqdm(ws):
        d = ds[w]
        d1 = d[1]
        if d1 == 0:
            n0 += 1
        elif d1 == 1:
            n1 += 1
        tot += 1
    assert tot == n0 + n1
    print(n0/tot)
    print(n1/tot)

In [24]:
arch=resnet34
data = ImageClassifierData.from_paths(PATH, tfms=tfms_from_model(arch, sz), bs=16)

In [28]:
y.shape[0]

8

In [31]:
c0 = 0
c1 = 0
tot = 0
for x, y in tqdm(data.trn_dl):
    ys = y.shape[0]
    num1 = torch.sum(y)
    c0 += ys - num1
    c1 += num1
    tot += ys
assert tot == c0 + c1
print(c0 / tot)
print(c1 / tot)

100%|██████████| 1438/1438 [00:16<00:00, 89.68it/s]
0.5
0.5


In [32]:
ds = data.trn_dl.dataset

In [35]:
def check_sampler(dl):
    c0 = 0
    c1 = 0
    tot = 0
    for x, y in tqdm(dl):
        ys = y.shape[0]
        num1 = torch.sum(y)
        c0 += ys - num1
        c1 += num1
        tot += ys
    assert tot == c0 + c1
    print(c0 / tot)
    print(c1 / tot)

In [36]:
check_sampler(data.trn_dl)

100%|██████████| 1438/1438 [00:16<00:00, 89.75it/s]
0.5
0.5


In [81]:
new_dl = DataLoader(ds, batch_size=16, sampler=WeightedRandomSampler([0.25, 0.75], len(ds), replacement=True), num_workers=4)

In [50]:
len(ds)

23000

In [136]:
wl2i = []

l2p = [0.25, 0.75]

for ind, d in enumerate(tqdm(ds)):
    wl2i.append(l2p[d[1]])

100%|██████████| 23000/23000 [01:22<00:00, 279.93it/s]


In [149]:
ws2 = WeightedRandomSampler(wl2i, len(ds)//2, replacement=True)

In [150]:
check_ws(ws2, ds)

100%|██████████| 11500/11500 [00:42<00:00, 273.25it/s]
0.25417391304347825
0.7458260869565217


In [142]:
list(ws2)

[13984,
 9559,
 14438,
 21993,
 20024,
 9415,
 19728,
 21066,
 19997,
 22961,
 2496,
 9397,
 20366,
 17331,
 12219,
 18241,
 12593,
 4029,
 20921,
 14357,
 22688,
 5786,
 18997,
 21537,
 20197,
 10976,
 11957,
 20817,
 21251,
 13360,
 15805,
 8517,
 9496,
 21214,
 18583,
 16485,
 22835,
 15537,
 11665,
 1057,
 13726,
 3712,
 13933,
 12827,
 19553,
 16911,
 18194,
 10907,
 5587,
 16399,
 16638,
 14332,
 18759,
 95,
 12150,
 6922,
 4696,
 18083,
 18695,
 16658,
 15046,
 18663,
 12227,
 13276,
 22489,
 14276,
 10005,
 11656,
 3074,
 3126,
 6332,
 1814,
 19788,
 12417,
 3581,
 16031,
 408,
 20499,
 22335,
 17907,
 19527,
 19357,
 9870,
 16641,
 11601,
 15077,
 19977,
 8306,
 21472,
 14515,
 18599,
 3943,
 19674,
 14553,
 16637,
 20534,
 6449,
 13327,
 16884,
 21241,
 8555,
 14899,
 6685,
 4798,
 21429,
 15369,
 13441,
 1065,
 17692,
 22801,
 20682,
 19218,
 14270,
 15241,
 17837,
 1418,
 12704,
 19432,
 19905,
 14950,
 19671,
 21394,
 3917,
 4499,
 22242,
 16046,
 20112,
 15027,
 14733,
 1

In [112]:
a1 = ws2